# 📝 EfficientDet Training

## requirement 수행

In [1]:
# 학습 진행 파일이 있는 경로로 이동
%cd automl/efficientdet

/aiffel/aiffel/EfficientDet/automl/efficientdet


In [2]:
# requirement 설치
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-sk5_ipjw
  Running command git clone --filter=blob:none -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-sk5_ipjw
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 588.3 MB 9.1 kB/s              
     |████████████████████████████████| 1.1 MB 57.5 MB/s            
     |████████████████████████████████| 128 kB 88.9 MB/s            
     |████████████████████████████████| 2.4 MB 72.1 MB/s            
     |████████████████████████████████| 14.1 MB 72.1 MB/s            
     |████████████████████████████████| 124 kB 83.3 MB/s            
     |████████████████████████████████| 6.0 MB 74.1 MB/s            
     |████████████████████████████████| 1.7 MB 92.5 MB/s            
     |████████████████████████████████| 439 kB 83.6 MB/s            


  Created wheel for pycocotools: filename=pycocotools-2.0-cp39-cp39-linux_x86_64.whl size=103741 sha256=2f01babdc49341d56e30d272c1f20585b2767c46f277cc378ec31b63ba8876a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-93xkj69j/wheels/13/c1/d6/a321055f7089f1a6af654fbf794536b196999f082a9cb68a37
Successfully built pycocotools
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled

In [3]:
# wandb 설치
!pip install wandb

     |████████████████████████████████| 1.9 MB 6.0 MB/s            
     |████████████████████████████████| 182 kB 71.1 MB/s            
     |████████████████████████████████| 168 kB 99.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 62 kB 2.6 MB/s             
     |████████████████████████████████| 140 kB 82.3 MB/s            
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=1cea40c39eb355fa31063b3da7436e097e1bd7070e6ab0f7c8c2ca68a554e7a8
  Stored in directory: /aiffel/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7


## 학습 진행할 모델 pretrained_ckpt 다운로드

In [4]:
import os

# 학습할 모델
MODEL = 'efficientdet-d2' 

def download(m):
    if m not in os.listdir():
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
        !tar zxf {m}.tar.gz
    ckpt_path = os.path.join(os.getcwd(), m)
    return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))

Use model in /aiffel/aiffel/EfficientDet/automl/efficientdet/efficientdet-d2


## 모델 config parameter 업데이트

In [5]:
# config parameter 작성
class TRAIN_CFG:
    model_name = 'efficientdet-d2' # efficientdet 모델명
    strategy = '' # tpu, 여러개의 GPU들, 단일 GPU 일때 학습 strategy 설정. 
    model_dir = '/aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt' # 학습된 모델이 저장될 위치
    pretrained_ckpt = '/aiffel/aiffel/EfficientDet/automl/efficientdet/efficientdet-d2' # coco로 pretrained된 checkpoint 파일이 있는 디렉토리 위치
    pretrain_or_ckpt = 'False'     # True 면 pretrain_ckpt, False 면 이전의 ckpt 
    hparams = 'num_classes=4,moving_average_decay=0,mixed_precision=true' # 하이퍼파라미터 설정
    use_xla = False
    use_fake_data = False
    # max_instances_per_image를 200으로 설정하고, D2 모델이 상대적으로 더 크기 때문에 batch_size를 8로 설정시 GPU Out of Memory 발생 
    batch_size = 4
    eval_samples = 2000 # evaluation image 데이터 갯수
    steps_per_execution = 1 # train 시 steps 횟수
    num_examples_per_epoch = 50 # 1 epochs 시 적용하는 examples 개수 
    num_epochs = 100 # epochs 횟수
    train_file_pattern = '/aiffel/aiffel/AIFFELTHON/real_dataset/tfrecord/train/train-*.tfrecord' # 학습용 tfrecords를 glob 형태로 가져오는 표현식. 
    val_file_pattern = '/aiffel/aiffel/AIFFELTHON/real_dataset/tfrecord/val/val-*.tfrecord' # 검증용 tfrecords를 glob 형태로 가져오는 표현식. 
    val_json_file = None # optional coco validation json 
    mode = 'traineval' # train만 적용 또는 train과 eval함께 적용(traineval)
    num_shards = 100 
    max_instances_per_image = 200
    image_size = '960x512'
    label_map = {
    'car' : 0, 
    'truck' : 1, 
    'bus' : 2, 
    'Pedestrian' : 3}
    
    num_cores = 2 # tpu 일때 8 적용.  
    tpu = None
    gcp_project = None
    tpu_zone = None
    eval_master = ''
    eval_name = None
    tf_random_seed = 2021
    profile = False
    debug = False

In [6]:
# 위에서 작성한 config parameter들을 업데이트
from tf2.train import setup_model
import hparams_config

import utils
# from tf2 import tfmot
from tf2 import train_lib
from tf2 import util_keras
import tensorflow as tf

config = hparams_config.get_detection_config(TRAIN_CFG.model_name)
config.override(TRAIN_CFG.hparams)

steps_per_epoch = TRAIN_CFG.num_examples_per_epoch // TRAIN_CFG.batch_size

if tf.config.list_physical_devices('GPU'):
    ds_strategy = tf.distribute.OneDeviceStrategy('device:GPU:0')
else:
    ds_strategy = tf.distribute.OneDeviceStrategy('device:CPU:0')

print(ds_strategy)

params = dict(
      profile=TRAIN_CFG.profile,
      mode = TRAIN_CFG.mode,
      model_name=TRAIN_CFG.model_name,
      steps_per_execution=TRAIN_CFG.steps_per_execution,
      num_epochs = TRAIN_CFG.num_epochs,
      model_dir=TRAIN_CFG.model_dir,
      steps_per_epoch=steps_per_epoch,
      strategy=TRAIN_CFG.strategy,
      batch_size=TRAIN_CFG.batch_size,
      tf_random_seed=TRAIN_CFG.tf_random_seed,
      debug=TRAIN_CFG.debug,
      val_json_file=TRAIN_CFG.val_json_file,
      eval_samples=TRAIN_CFG.eval_samples,
      num_shards=ds_strategy.num_replicas_in_sync,
      max_instances_per_image = TRAIN_CFG.max_instances_per_image,
      image_size= TRAIN_CFG.image_size,
      pretrain_or_ckpt=TRAIN_CFG.pretrain_or_ckpt,
      label_map=TRAIN_CFG.label_map
      )

config.override(params, True)

# image size를 tuple 형태로 변환. 512는 (512, 512)로 '1920x880' 은 (1920, 880) 으로 변환.  
config.image_size = utils.parse_image_size(config.image_size)
print(config)

act_type: swish
alpha: 0.25
anchor_scale: 4.0
apply_bn_for_resampling: true
aspect_ratios:
- 1.0
- 2.0
- 0.5
autoaugment_policy: null
backbone_config: null
backbone_name: efficientnet-b2
batch_size: 4
box_class_repeats: 3
box_loss_weight: 50.0
ckpt_var_scope: null
clip_gradients_norm: 10.0
conv_after_downsample: false
conv_bn_act_pattern: false
data_format: channels_last
dataset_type: null
debug: false
delta: 0.1
drop_remainder: true
eval_samples: 2000
first_lr_drop_epoch: 200.0
fpn_cell_repeats: 5
fpn_config: null
fpn_name: null
fpn_num_filters: 112
fpn_weight_method: null
gamma: 1.5
grad_checkpoint: false
grid_mask: false
heads:
- object_detection
image_size: !!python/tuple
- 512
- 960
img_summary_steps: null
input_rand_hflip: true
iou_loss_type: null
iou_loss_weight: 1.0
is_training_bn: true
jitter_max: 2.0
jitter_min: 0.1
label_map:
    Pedestrian: 3
    bus: 2
    car: 0
    truck: 1
label_smoothing: 0.1
learning_rate: 0.08
loss_scale: null
lr_decay_method: cosine
lr_warmup_epoch:

## wandb 로그인 및 설정

In [7]:
import wandb

wandb.login()

run = wandb.init(project = 'efficinetdet-d2',
                 entity = '',
                 name = 'h_default_epoch_100')

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc
wandb: Currently logged in as: minkikwak. Use `wandb login --relogin` to force relogin


## 학습할 데이터 준비

In [8]:
import dataloader

def get_dataset(is_training, config):
    # is_training이 True이면 TRAIN_CFG의 train_file_pattern, 그렇지 아니면 val_file_pattern
    file_pattern = (
      TRAIN_CFG.train_file_pattern
      if is_training else TRAIN_CFG.val_file_pattern)
    
    if not file_pattern:
        raise ValueError('No matching files.')
    
    return dataloader.InputReader(
    file_pattern,
    is_training=is_training,
    use_fake_data=TRAIN_CFG.use_fake_data,
    max_instances_per_image=config.max_instances_per_image,
    debug=TRAIN_CFG.debug)(
        config.as_dict())

## 학습 진행할 모델 준비

In [9]:
from tf2 import train_lib
from tf2 import train

def get_efficientdet_model(config):
    # 2개의 class를 가진 efficientdet d2 모델을 생성. 
    model = train_lib.EfficientDetNetTrain(config=config)
    model = train.setup_model(model, config)
    
    # pretrain_ckpt 불러오기
    if config.pretrain_or_ckpt == 'True':
        ckpt_path = tf.train.latest_checkpoint(TRAIN_CFG.pretrained_ckpt)
        # classfication layer를 제외하고 pretrained weight를 생성된 모델로 로딩. 
        util_keras.restore_ckpt(
                model,
                ckpt_path,
                config.moving_average_decay,
                exclude_layers=['class_net'])
        print('pretrain ckpt 불러오기!!!')
        
    # 이전 ckpt 불러오기    
    else:
        ckpt_path = tf.train.latest_checkpoint(TRAIN_CFG.model_dir)
        util_keras.restore_ckpt(
                model,
                ckpt_path,
                config.moving_average_decay)
        print('이전 ckpt 불러오기!!!')
    train.init_experimental(config)
    return model

model = get_efficientdet_model(config)
model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/tmp/__autograph_generated_fileoe4bu57h.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  ag__.for_stmt(ag__.ld(self).updates, None, loop_body, get_state, set_state, (), {'iterate_names': 'u'})
/aiffel/aiffel/EfficientDet/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:


이전 ckpt 불러오기!!!
Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b2 (Model)     multiple                  7267314   
                                                                 
 resample_p6 (ResampleFeatur  multiple                 39984     
 eMap)                                                           
                                                                 
 resample_p7 (ResampleFeatur  multiple                 0         
 eMap)                                                           
                                                                 
 fpn_cells (FPNCells)        multiple                  678479    
                                                                 
 class_net (ClassNet)        multiple                  52788     
                                                                 
 box_net (BoxNet)            multiple             

In [10]:
print(config.mode)

traineval


## 학습 진행

In [ ]:
from tf2 import train
import numpy as np
import keras

config.batch_size = 4
tr_steps_per_epoch = 8000//config.batch_size # batch_size에 따른 epoch 당 train 데이터 수
val_steps_per_epoch = 2000//config.batch_size # batch_size에 따른 epoch 당 val 데이터 수
print('tr_steps_per_epoch:', tr_steps_per_epoch, 'val_steps_per_epoch:', val_steps_per_epoch)

# wandb loss값 저장
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        wandb.log(logs)

custom_hist = CustomHistory()
custom_hist.init()

val_dataset = get_dataset(False, config)

# pretrain_ckpt를 가져올지 이전의 ckpt를 가져올지 정함.
ckpt_path = tf.train.latest_checkpoint(TRAIN_CFG.model_dir)
if config.pretrain_or_ckpt == 'True':
    initial_epoch = 0
else:
    initial_epoch = int(os.path.basename(ckpt_path).split('-')[1])
    
# 학습 시작
model.fit(
    get_dataset(True, config),
    epochs=config.num_epochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=tr_steps_per_epoch,
    callbacks=[train_lib.get_callbacks(config.as_dict(), val_dataset),custom_hist],
    validation_data=val_dataset,
    validation_steps=val_steps_per_epoch)

tf.keras.backend.clear_session()

tr_steps_per_epoch: 2000 val_steps_per_epoch: 500
Epoch 76/100
2000/2000 [==============================] - ETA: 0s - det_loss: 0.1156 - cls_loss: 0.0499 - box_loss: 0.0013 - reg_l2_loss: 0.0842 - loss: 0.1998 - learning_rate: 0.0021 - gradient_norm: 1.6668
Epoch 76: saving model to /aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt/ckpt-76
2000/2000 [==============================] - 1303s 597ms/step - det_loss: 0.1156 - cls_loss: 0.0499 - box_loss: 0.0013 - reg_l2_loss: 0.0842 - loss: 0.1998 - learning_rate: 0.0021 - gradient_norm: 1.6669 - val_det_loss: 0.2086 - val_cls_loss: 0.0932 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0841 - val_loss: 0.2927
Epoch 77/100
2000/2000 [==============================] - ETA: 0s - det_loss: 0.1204 - cls_loss: 0.0528 - box_loss: 0.0014 - reg_l2_loss: 0.0840 - loss: 0.2044 - learning_rate: 0.0021 - gradient_norm: 1.7377
Epoch 77: saving model to /aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt/ckpt-77
2000/2000 [==============================] - 1186s

2000/2000 [==============================] - 1277s 639ms/step - det_loss: 0.1209 - cls_loss: 0.0527 - box_loss: 0.0014 - reg_l2_loss: 0.0829 - loss: 0.2038 - learning_rate: 0.0028 - gradient_norm: 1.7128 - val_det_loss: 0.2088 - val_cls_loss: 0.1006 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0829 - val_loss: 0.2917
Epoch 86/100
2000/2000 [==============================] - ETA: 0s - det_loss: 0.1199 - cls_loss: 0.0518 - box_loss: 0.0014 - reg_l2_loss: 0.0828 - loss: 0.2027 - learning_rate: 0.0029 - gradient_norm: 1.6774
Epoch 86: saving model to /aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt/ckpt-86
2000/2000 [==============================] - 1187s 593ms/step - det_loss: 0.1199 - cls_loss: 0.0518 - box_loss: 0.0014 - reg_l2_loss: 0.0828 - loss: 0.2027 - learning_rate: 0.0029 - gradient_norm: 1.6773 - val_det_loss: 0.1893 - val_cls_loss: 0.0837 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0827 - val_loss: 0.2720
Epoch 87/100
2000/2000 [==============================] - ETA: 0s - det_l

2000/2000 [==============================] - 1277s 638ms/step - det_loss: 0.1149 - cls_loss: 0.0499 - box_loss: 0.0013 - reg_l2_loss: 0.0816 - loss: 0.1964 - learning_rate: 0.0021 - gradient_norm: 1.7061 - val_det_loss: 0.1874 - val_cls_loss: 0.0892 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0815 - val_loss: 0.2689
Epoch 96/100
2000/2000 [==============================] - ETA: 0s - det_loss: 0.1157 - cls_loss: 0.0491 - box_loss: 0.0013 - reg_l2_loss: 0.0814 - loss: 0.1971 - learning_rate: 0.0022 - gradient_norm: 1.7239
Epoch 96: saving model to /aiffel/aiffel/EfficientDet/efficientdet-d2_ckpt/ckpt-96
2000/2000 [==============================] - 1188s 594ms/step - det_loss: 0.1157 - cls_loss: 0.0491 - box_loss: 0.0013 - reg_l2_loss: 0.0814 - loss: 0.1971 - learning_rate: 0.0022 - gradient_norm: 1.7246 - val_det_loss: 0.2159 - val_cls_loss: 0.1073 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0814 - val_loss: 0.2972
Epoch 97/100
2000/2000 [==============================] - ETA: 0s - det_l